# Introduction au Deep Learning avec Theano 

 - Julien Guillaumin 
 - Etudiant à Télécom Bretagne en année de césure 
 - Stage chez Continental (Toulouse) - Deep Learning (octobre 2016 à mars 2017)
 - Kaggle-addict et MOOC-friendly 

Objectifs de l'atelier :
- découvrir le Deep Learning et les différences avec le Machine Learning "non profond"
- l'appliquer à la classification d'images [détection d'anomalie]
- le tout en découvrant `thenao`

1. Rappels de Machine Learning 
2. Découvrir Theano
3. Régression logistique pour la classification 
4. Réseaux de neurones, un premier pas dans le Deep Learning 
5. Réseaux convolutifs, de 1990 à 2012 
6. [La détection d'anomalies par entrainement non-supérvisé !]

# I/ Rappels de Machine Learning

- Champ d'étude de l'intelligence artificielle 
- Concevoir des algorithmes qui adaptent leurs réponses en fonction de l'environnement/comportement dans lequel ils se trouvent 
- **Etape d'apprentissage** : on montre des exemples de comportements -> nos **données**
    * l'agorithme doit **ajuster ses paramètres** pour produire la réponse voulue, **minimiser une erreur**
    * **Généralisation** et non un apprentissage par coeur 
- **Etape d'inference** : on met notre algorithme dans un nouvel environnement où il doit produire des réponses correctes

## Apprentissage supervisé vs Apprentissage non-supervisé 

* **Apprentissage supervisé : on connaît la bonne réponse !** 
  - Algorithmes de classification : estimer la probabilté que cette données appartienne à telle classe
  - Algorithmes de régression : prédire une valeur réelle à partir des données 


* **Apprentissage non-supervisé : trouver une structure/schéma dans les données !**
    - Algorithme de Clustering (segmentation de clients, analyse de l'ADN ou de données spatiales)

Aujourd'hui : 
- beaucoup d'apprentissage supervisé pour la classification d'images 
- un peu de non supervisé pour de la détection d'anomalie 


### Apprentissage supervisé 
Exemples de données : 

<img src="resources/imagenet.jpeg" width="900">



* Le i-ème exemple :   $( x^{(i)}, y^{(i)} )$
* Les paramètres entraînables du système :   $ \theta $
* Fonction de prédiction (la sortie de notre algorithme) :   $ f_{\theta}(x^{(i)})$



On parle aussi de fonction de coût, de perte.

Erreur pour un exemple : $$ E( x^{(i)} , y^{(i)} , \theta ) = L( f_{\theta}( x^{(i)} ) ,y^{(i)}) $$

Erreur pour un ensemble de données : $$ E( X, Y, \theta) = \frac{1}{N}\sum_{n=1}^N E( x^{(i)} , y^{(i)} , \theta ) $$

#### Pour la régression : 
 * $f_{\theta}(x^{(i)}) = \hat y$ est une approximation de $y^{(i)}$
 * on peut utiliser l'_Erreur Moyenne Quadratique_ (**MSE**) comme mesure de l'erreur ! 
$$ L( f_{\theta}( x^{(i)} ) ,y^{(i)}) = \frac{1}{2}|| f_{\theta}(x^{(i)}) - y^{(i)} ||^2 $$

#### Pour la classification : 
* $f_{\theta}(x^{(i)}) $ est une estimation de $\Bbb P( \ Y \ = \ i \ | \ x^{(i) \ } , \theta)$ pour toute classe $i$ (vecteur de probabilités)
* **-** log-vraisemblance (Negative Log Likelihood): 

$$  L( \ f_{\theta}( x^{(i)}) \ , \ y^{(i)} \ ) = - \ log \ ( P( \ Y \ = \ y^{(i)} \ | \ x^{(i) \ } , \theta) ) $$
  

Nous sommes revenus à un problème d'optimisation : 
 * **Trouver $\theta$ qui minimise $E( X, Y, \theta)$**

<img src="resources/gradient.png" width="500">


**Descente du gradient** :
- Algorithme itératif 
- $ \lambda $ pas d'apprentissage 
- Mettre à jour $\theta$ de façon à se rapprocher du minimum global

$$ \theta_{t+1} = \theta_{t} \ - \ \lambda \ \frac{\partial E(X,Y,\theta)}{\partial \theta}$$

    

# II/ Theano

<img src="resources/theano_logo.png" width="500">


* Librairie Python pour définir et évaluer des expressions mathématiques 
* Représentation d'une expression par un graphe 
* Générer et compiler du code optimisé pour CPU (C/C++) ou GPU (CUDA)
* CPU/GPU transparent pour l'utilisateur
* Dérivation automatique (très utile pour le gradient)
* Syntaxe proche de Numpy


## Définir une expression 
Un exemple d'expression mathématique : $$c = \sqrt{a^2 + b^2}$$

3 variables symboliques: $$a, b , c$$
(scalaires, vecteurs, matrices, tenseur 3D, 4D, ...) 



In [1]:
import theano
import theano.tensor as T 
# on y retrouve toutes les variables symboliques et de nombreuses opérations. 

x = T.matrix('x')
y = T.matrix('y')

z = x + y

Using gpu device 0: GeForce GTX 670MX (CNMeM is enabled with initial size: 10.0% of memory, cuDNN 5105)


<img src="resources/applyAdditionMatrix.png" width="300">

## Réprésentation sous forme d'un graphe

<img src="resources/symbolic_graph_unopt.png" width="600">

## Compiler l'expression et l'utiliser comme une fonction

In [2]:
#compilation de l'expression 'c' sous forme d'une fonction 'f'
#le code généré puis compilé est gardé en mémoire 
f = theano.function([x,y],z)

#On peut directement appeler la function 'f'
f( [[0,1],[2,8]] , [[2,5],[-4,5]] )

array([[  2.,   6.],
       [ -2.,  13.]], dtype=float32)

<img src="resources/symbolic_graph_opt.png" width="600">

### Optimisation 
- génération de code optimisé pour les opérations et variables de l'expression 
- CPU ou GPU (indifférent pour l'utilisateur)
- optimisation dans la représentation de l'expression !

```export THEANO_FLAGS='cuda.root=/usr/local/cuda-7.5,device=gpu,floatX=float32'```

* GPU : x 10 sur ma machine
* CUDA, cuDNN, cuBLAS, CNMeM ... 

![alt text](resources/nvidia.png "Title")

# Dérivation dans Thenao

- Dérivation automatique d'une expression 
- Application de la règle de dérivation des fonctions composées ! 

$$ y(x) = x^2$$
$$ \ $$
$$ \frac{dy}{dx} = 2x $$

In [3]:
x = T.scalar('x')
y = x**2
# T.grad( expression scalaire, variable )
g_y = T.grad(y,x)

<img src="resources/g_y_unopt.png" width="600">

In [4]:
f = theano.function([x], g_y)
f(4)

array(8.0, dtype=float32)

<img src="resources/g_y_opt.png" width="600">

$$f(x)=k \ x^n$$
$$f'(x)=k \ n \ x^{n-1}$$

 * On passe au premier notebook sur Theano (avec un exemple de régression linéaire)

# Réseaux de Neurones, le début du Deep Learning

** Deep Learning :**
* On applique plusieurs non-linéarités à $x^{(i)}$ dans  $ f_{\theta}(x^{(i)})$
* On distingue des couches de non-linéarités


** Profondeur d'une architecture : **
* nombre de couches où des non-linéarités sont appliquées 

Avantages des architectures profondes :
* inspiration du système visuel animal
* extraction de caractéristiques à plusieurs niveaux d'abstarction (pratique)
* représentation efficace de fonctions complexes (théorique)

Inconvénients : 
* l'entrainement devient vite très long

Tâches : classification, régression, réduction de dimensionalité, segmentation d'images, génération d'images, etc.

## Perceptron Multicouche 
* Réseau de neurones artificiels (dès 1980)
* Facilité pour empiler les couches 

<img src="resources/mlp1.png" width="600">



<img src="resources/mlp2.png" width="600">

## Apprentissage : Rétropropagation du gradient 


- Initilisation des poids du réseau (règle de "Xavier" par exemple)
- Pour un exemple donné, on effectue la propagation avant
- Calcul de l'erreur en sortie (par rapport à une valeur cible)
- Calcul du signal d'erreur sur les couches cachées par rétropropagation du gradient 
- Correction des poids 


## Difficultés d'entraîner des modèles profonds 
* Le nombre de paramètres croît très vite (...Bon dimensionnement du réseau) 
* Mauvaise diffusion du gradient vers les couches basses (...Bonne initialisation)
* Fonction à minimiser fortement non-convexe (... Techniques de régularisation)

# Réseaux convolutifs 

* Tirer profit de la forte corrélation entre pixels voisins, pour limiter le nombre de paramètres
<img src="resources/sparse_1D_nn.png" width="500">

* Partager des poids, pour limiter le nombre de paramètres 
<img src="resources/conv_1D_nn.png" width="800">

On retrouve tout ça dans la convolution : 
* On déplace sur l'image un noyau de convolution dont les poids sont les paramètres entraînables
<img src="resources/kernel_convolution.jpg" width="500">

On crée ainsi des <b>Feature Maps<b>
<img src="resources/Conv-Nets.png" width="800">

Ici 4 noyaux 3D, pour passer d'une couche avec 3 feature maps à une couche avec 4 feature maps
<br>(3x3x3) x 4 -> 108 paramètres

* Pooling (max ou moyenne) 
* Fonction d'activation à chaque pixel des feature maps

<img src="resources/pooling.png" width="600">

<img src="resources/Conv_full_model.png" width="1400">

<img src="resources/mylenet.png" width="1400">